# Plots
Code required to recreate graphs in *Optimization of Liquid Handling Parameters for Viscous Liquid Transfers with Pipetting Robots, a “Sticky Situation”* [(10.26434/chemrxiv-2023-cbkqh)](https://doi.org/10.26434/chemrxiv-2023-cbkqh)

## Imports

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

import plotly.colors
from plotly.subplots import make_subplots
import re
import plotly.offline as pyo
import plotly.graph_objects as go
pyo.init_notebook_mode()

import plotly.io as pio
import os
pio.templates.default = 'seaborn'




## Bar chart graphs

Load  csv containing the summary of the transfer errors and standard deviations obtained for Opentrons and Sartorius pipettes 

In [ ]:
dfs = {}

dfs['Sartorius'] = pd.read_csv(os.getcwd().split('Plots')[0]+r'Sartorius/Summary_Sartorius_best_parameters_statistics.csv')
dfs['Sartorius']

In [ ]:
dfs['Opentrons'] = pd.read_csv(os.getcwd().split('Plots')[0]+r'Opentrons/Summary_Opentrons_best_parameters_statistics.csv')
dfs['Opentrons'] 

In [ ]:
def compare_pipettes(y,driver='ML',range=[0,7.5],with_line=[5],marker_color1='#7B68EE',marker_color2='#B22222'):
    """
    This functions creates a bar chart that compares two separate pipettes for a specific metric and procedure using the pandas.DataFrame defined in dfs dictionary.
    Args:
        y (str): Name of column in dataFrame that contains the data that will be compared 
        driver (str): Value in the driver column in the DataFrame that specifies if the liquid handling parameters to be compared were obtained 
        by ML algorithm or human intuition. Select 'ML' or 'Human'.
        range (list): List containing minimum and maximum values in y axis
        with_line (list): List of values where a horizontal dotted line is required
        marker_color1 (str): Defines color for Opentrons bar chart
        marker_color2 (str):  Defines color for Sartorius bar chart
    """
    ot2_df =dfs['Opentrons'] 
    sartorious_df= dfs['Sartorius'] 
    
    if len(y.split(' ',-1))<7:
        title_y = y
    else:  
        title_y = y.split(' ',-1)
        title_y.insert(4,'<br>')
        title_y = ' '.join(title_y)
    
    ot2 = ot2_df.where(ot2_df['Driver']==driver).dropna(how='all').copy()
    sartorious = sartorious_df.where(sartorious_df['Driver']==driver).dropna(how='all').copy()

    layout = dict(xaxis = dict(showgrid=False,mirror=True,ticks='outside',showline=True, tickfont=dict(size=20)),
        yaxis = dict(showgrid=False,mirror=True,ticks='outside',showline=True,tickfont=dict(size=20))
        )

    fig = go.Figure(
        data=[go.Bar(name = 'Opentrons',x =[str(int(x))+' cp' for x in ot2['Viscosity'].to_list()], 
                    y = ot2[y], marker_color=marker_color1), 
            go.Bar(name = 'Sartorius',x = [str(int(x))+' cp' for x in sartorious['Viscosity'].to_list()], 
                    y = sartorious[y], marker_color=marker_color2)], layout = layout)

    fig.update_layout(xaxis_title='Viscosity Standards', yaxis_title=title_y, 
                      width = 600, height = 460,
                      plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0,0,0,0)',showlegend=False)

    fig.add_hline(y=0,line_dash='solid',opacity=1,line_width=2)
    if with_line!=None:
        colors = ['green','#820970','#820970']
        for index,value in enumerate(with_line):
            if value !=0:
                fig.add_hline(y=value,line_dash="dash", line_color=colors[index], line_width=3,opacity=1)
                fig.add_hline(y=-value,line_dash="dash", line_color=colors[index], line_width=3, opacity=1)

    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', title_font=dict(size=24))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', range = range,title_font=dict(size=24))
    return fig

In [ ]:
driver = 'Human' #input driver Human or ML
folder = #add path to save image

In [ ]:
x =compare_pipettes('Mean percentage error for transfer of 1000 µL [%]',driver,range=[-7.5,7.5],with_line=[5,0.7])
x.write_image(f'{folder}{driver}_e_1000.svg')
x

In [ ]:
x=compare_pipettes('Percentage standard deviation for transfer of 1000 µL [%]',driver, range=[0,4.5],with_line=[0,0.15])
x.write_image(f'{folder}{driver}_std_1000.svg')
x

In [ ]:
x=compare_pipettes('Mean percentage error for transfer of 500 µL [%]',driver,range=[-7.5,7.5],with_line=[5,1])
x.write_image(f'{folder}{driver}_e_500.svg')
x

In [ ]:
x=compare_pipettes('Percentage standard deviation for transfer of 500 µL [%]',driver, range=[0,4.5],with_line=[0,0.2])
x.write_image(f'{folder}{driver}_std_500.svg')
x

In [ ]:
x=compare_pipettes('Mean percentage error for transfer of 300 µL [%]',driver,range=[-7.5,7.5],with_line=[5,1,2])
x.write_image(f'{folder}{driver}_e_300.svg')
x



In [ ]:
x=compare_pipettes('Percentage standard deviation for transfer of 300 µL [%]',driver, range=[0,4.5],with_line=[0,0.2,1])
x.write_image(f'{folder}{driver}_std_300.svg')
x

In [ ]:
x=compare_pipettes('Iterations to find best parameter', driver,range=[0,15], with_line=[0])
x.write_image(f'{folder}{driver}_iterations.svg')
x


In [ ]:

x=compare_pipettes('Time to aspirate 1000 µL [s]',driver,range=[0,650],with_line=[0])
x.write_image(f'{folder}{driver}_time.svg')
x

In [ ]:
def compare_techniques(df,y,techniques=['Human','ML'],colours=['#FF5733','#6495ED'],range=[0,7.5],with_line=[0]):
    """
    This functions creates a bar chart that compares  a calibration metric for two different optimization techniques using one specific pipette.
    Args:
        df (pandas.DataFrame): dataframe containig the summary of the results for a specific pipette. 
        y (str): Name of column in dataFrame that contains the data that will be compared 
        techniques (list): List of strings containing values of driver column in the DataFrame that specifies if the liquid handling parameters to be compared were obtained 
        by ML algorithm or human intuition. Select a value or combination of 'ML'(semi-automated algorithm),'FA' (fully automated algorithm) or 'Human'.
       colours (list): Defines list of color names (str) that will be used to compare between dataFrames.
        range (list): List containing minimum and maximum values in y axis
        with_line (list): List of values where a horizontal dotted line is required  
    """
    if len( y.split(' ',-1))<7:
        title_y = y
    else:  
        title_y = y.split(' ',-1)
        title_y.insert(4,'<br>')
        title_y = ' '.join(title_y)
    layout = dict(xaxis = dict(showgrid=False,mirror=True,ticks='outside',showline=True, tickfont=dict(size=20)),
                yaxis = dict(showgrid=False,mirror=True,ticks='outside',showline=True,tickfont=dict(size=20))
                )
  
    fig = go.Figure(layout=layout)

    for index,technique in enumerate(techniques):
        fig.add_bar(name = technique,x =[str(int(x))+' cp' for x in df.where(df['Driver']==technique).dropna(how='all')['Viscosity'].to_list()], 
                        y = df.where(df['Driver']==technique).dropna(how='all')[y], marker_color=colours[index])

    fig.update_layout(xaxis_title='Viscosity Standards', yaxis_title=title_y,
                      width = 600, height = 460,
                      plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0,0,0,0)',showlegend=False)
    fig.add_hline(y=0,line_dash='solid',opacity=1,line_width=2)
    if with_line!=None:
        colors = ['green','#820970','#820970']
        for index,value in enumerate(with_line):
            if value !=0:
                fig.add_hline(y=value,line_dash="dash", line_color=colors[index], line_width=3, opacity=1)
                fig.add_hline(y=-value,line_dash="dash", line_color=colors[index], line_width=3, opacity=1)

    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', title_font=dict(size=24))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', range = range,title_font=dict(size=24))
    return fig


In [ ]:
df ='Opentrons' #input equipment: Opentrons or Sartorius
techniques = ['Human','ML'] #input drivers to be compared: Human, ML or FA
colours=['#6495ED','#FF5733','#C9292D'] # input list of colours to be used
comparison = '_vs_'.join(techniques)
folder = '' #input path to folder where image will be saved

In [ ]:
x=compare_techniques(dfs[df],'Mean percentage error for transfer of 1000 µL [%]',techniques=techniques,colours=colours,range=[-7.5,7.5],with_line=[5,0.7])
x.write_image(f'{folder}{df}{comparison}_e_1000.svg')
x

In [ ]:
x=compare_techniques(dfs[df],'Percentage standard deviation for transfer of 1000 µL [%]',techniques=techniques,colours=colours, range=[0,5],with_line=[0,0.15])
x.write_image(f'{folder}{df}{comparison}_std_1000.svg')
x

In [ ]:
x=compare_techniques(dfs[df],'Mean percentage error for transfer of 500 µL [%]',techniques=techniques,colours=colours,range=[-7.5,7.5],with_line=[5,1])
x.write_image(f'{folder}{df}{comparison}_e_500.svg')
x

In [ ]:
x=compare_techniques(dfs[df],'Percentage standard deviation for transfer of 500 µL [%]', techniques=techniques,colours=colours,range=[0,5],with_line=[0,0.2])
x.write_image(f'{folder}{df}{comparison}_std_500.svg')
x

In [ ]:
x=compare_techniques(dfs[df],'Mean percentage error for transfer of 300 µL [%]',techniques=techniques,colours=colours,range=[-7.5,7.5], with_line=[5,1,2])
x.write_image(f'{folder}{df}{comparison}_e_300.svg')
x

In [ ]:
x=compare_techniques(dfs[df],'Percentage standard deviation for transfer of 300 µL [%]',techniques=techniques,colours=colours,range=[0,5],with_line=[0,0.2,1])
x.write_image(f'{folder}{df}{comparison}_std_300.svg')
x

In [ ]:
x=compare_techniques(dfs[df],'Iterations to find best parameter', techniques=techniques,colours=colours,range=[0,20])
x.write_image(f'{folder}{df}{comparison}_itreation.svg')
x

In [ ]:
x=compare_techniques(dfs[df],'Time to aspirate 1000 µL [s]',techniques=techniques,colours=colours,range=[0,900])
x.write_image(f'{folder}{df}{comparison}_time.svg')
x

## Initialization graph
The following code was used to generate the graph describing the automated initialization step (Figure 5)

In [ ]:
#code in this cell will load and process raw data to obtain filtered and fitted values of the mass vs time curve


from scipy.optimize import curve_fit
import os

def sigmoid(x, L ,x0, k,p,b):
    y = L / (1 + np.exp(k*(x-x0)))**(1/p) + b
    return (y)
mass_data = {}

folder = os.getcwd().split('Plots')[0]+'\\Sartorious\Auomated_initialization\\'
for i in os.listdir(folder):
    if 'Initialization' in i:
        mass_data[i.split('std')[1][1:-4]]= pd.read_csv(folder+i)

for key in mass_data.keys():
    mass_data[key]['Mass'] = mass_data[key]['Mass']-  mass_data[key]['Mass'][0]
    mass_data[key]['Mass_smooth'] = mass_data[key]['Mass_smooth']-  mass_data[key]['Mass_smooth'][0]
    mass_data[key]['ts'] = mass_data[key]['Time'].astype('datetime64[ns]').values.astype('float') / 10 ** 9
    mass_data[key]['ts']=mass_data[key]['ts']-mass_data[key]['ts'][0]

for key in mass_data.keys():  
    # mass_data[key] = mass_data[key].where(mass_data[key]['ts']>10).dropna()
    data_fit = mass_data[key].where(mass_data[key]['ts']>10).dropna()

    p0 = [min(data_fit['Mass']), np.median(data_fit['ts']),1,1,max(data_fit['Mass'])+30] # this is an mandatory initial guess
    # print(p0)

    popt, pcov = curve_fit(sigmoid, data_fit['ts'], data_fit['Mass'],p0)

    # print(popt)

    mass_sigmoid = sigmoid(data_fit['ts'],popt[0],popt[1],popt[2],popt[3],popt[4])
    mass_data[key].insert(len(mass_data[key].columns),'Mass_sigmoid', None)
    mass_data[key].loc[data_fit.index[0]:,'Mass_sigmoid'] = mass_sigmoid
    

In [ ]:
#code in this cell will make plot presented in Figure 5


fig = make_subplots(rows=2, cols=4,column_widths=[0.011]*4,horizontal_spacing=0.09)

colours= ['#1f77b4','#ff7f0e','#92D050']

column = 1
for key in ['204','1275']:
    plot_data = mass_data[key].dropna()
    fig.add_trace(
        go.Scatter(x=plot_data['ts'], y=plot_data['Mass'],mode='lines',marker_color=colours[0]),
        row=1, col=column
    )
    fig.add_trace(
        go.Scatter(x=plot_data['ts'], y=plot_data['Mass_smooth'],mode='lines',marker_color=colours[1]),
        row=1, col=column
    )

    if column>2:

        fig.add_trace(
            go.Scatter(x=plot_data['ts'], y=plot_data['Mass'].diff()/plot_data['ts'].diff(),mode='lines',marker_color=colours[0]),
            row=2, col=column
        )
        fig.add_trace(
            go.Scatter(x=plot_data['ts'], y=plot_data['Mass_derivative_smooth']/plot_data['ts'].diff(),mode='lines',marker_color=colours[1]),
            row=2, col=column
        )

    else:

        fig.add_trace(
            go.Scatter(x=plot_data['ts'], y=plot_data['Mass'].diff(),mode='lines',marker_color=colours[0]),
            row=2, col=column
        )
        fig.add_trace(
            go.Scatter(x=plot_data['ts'], y=plot_data['Mass_derivative_smooth'],mode='lines',marker_color=colours[1]),
            row=2, col=column
        )    
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',ticks='outside',showticklabels=False,mirror=True,row=1,col=column,tickfont= dict(size=14), titlefont=dict(size=18))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',ticks='outside',mirror=True,title_text= 'Mass<br>[mg]',row=1,col=column,tickfont= dict(size=14), titlefont=dict(size=18))
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',ticks='outside',mirror=True,title_text= 'Time<br>[s]',row=2,col=column,tickfont= dict(size=14), titlefont=dict(size=18))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',ticks='outside',mirror=True,title_text= 'dMass<br>[mg]',row=2,col=column,tickfont= dict(size=14), titlefont=dict(size=18))
    column+=1

for key in sorted(list(mass_data),reverse=True):
    plot_data = mass_data[key].dropna()
    fig.add_trace(
        go.Scatter(x=plot_data['ts'], y=plot_data['Mass'],mode='lines',marker_color=colours[0]),
        row=1, col=column
    )
    fig.add_trace(
        go.Scatter(x=plot_data['ts'].dropna(), y=plot_data['Mass_sigmoid'].dropna(),mode='lines',marker_color=colours[1]),
        row=1, col=column
    )

    if column>2:
        fig.add_trace(
            go.Scatter(x=plot_data['ts'], y=plot_data['Mass'].diff()/plot_data['ts'].diff(),mode='lines',marker_color=colours[0]),
            row=2, col=column
        )
        fig.add_trace(
        go.Scatter(x=plot_data['ts'], y=plot_data['Mass_sigmoid'].dropna().diff()/plot_data['ts'].diff(),mode='lines',marker_color=colours[1]),
        row=2, col=column
        )    
    else:
        fig.add_trace(
            go.Scatter(x=plot_data['ts'], y=plot_data['Mass'].diff(),mode='lines',marker_color=colours[0]),
            row=2, col=column
        )
        fig.add_trace(
            go.Scatter(x=plot_data['ts'], y=plot_data['Mass_sigmoid'].dropna().diff(),mode='lines',marker_color=colours[1]),
            row=2, col=column
        )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',ticks='outside',showticklabels=False,mirror=True,row=1,col=column)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',ticks='outside',mirror=True,title_text= 'Mass<br>[mg]',row=1,col=column,tickfont= dict(size=14), titlefont=dict(size=18))
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',ticks='outside',mirror=True,title_text= 'Time<br>[s]',row=2,col=column,tickfont= dict(size=14), titlefont=dict(size=18))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',ticks='outside',mirror=True,title_text= 'dMass/dTime<br>[mg/s]',row=2,col=column,tickfont= dict(size=14), titlefont=dict(size=18))
    column+=1

fig.update_layout(width = 1500, height = 700,
                  plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor = 'rgba(255,255,255,255)',showlegend=False)
fig.show()

## Iteration vs error and time
The following code was used to create the graphs where the percentage mean error and the time to transfer 10000 uL are plotted against transfer iteration.

In [1]:
#This cell defines functions used to create plots

import math

def calculate_means(df, features=['aspiration_rate','dispense_rate'],volume_list = [1000,500,300]):
    """
    This function is used to calculate the mean values for the transfers in each iteration of the optimization and the 
    time to aspirate a 1000 uL
    Args:
        df (pandas.DataFrame): DataFrame containing data from optimization 
        features (list): List of features names as str that were used to optimize the objectives 
        volume_list (list): List of volumes as int that were used during the optimization 
    """
    nan_columns = df.columns.to_list()
    nan_columns = [e for e in nan_columns if e not in ('liquid',
        'pipette',
        'volume',
        'aspiration_rate',
        'dispense_rate',
        'blow_out_rate',
        'delay_aspirate',
        'delay_dispense',
        'delay_blow_out',
        'touch_tip_aspirate',
        'touch_tip_dispense',
        'density',
        '%error',
        'time_asp_1000',
        'acq_value',
        'iteration')]
    
    if 'time_asp_1000' not in df.columns.to_list():
        df['time_asp_1000'] = 1000/df['aspiration_rate'] + 1000/df['dispense_rate'] + 20
    if 'acq_value' not in df.columns:
        df['acq_value'] = None

    counter =1 
    for row in df.drop_duplicates(['aspiration_rate','dispense_rate']).iterrows():
        df_repeat = df.loc[:,['aspiration_rate','dispense_rate']]== row[1].loc[['aspiration_rate','dispense_rate']]
        index = df_repeat.where(df_repeat.sum(axis=1)==2).dropna().index
        df.loc[index,'iteration']=counter
        counter+=1

    if df.loc[:,features].duplicated().sum()==0:
        df_mean = df
    else:
        df_duplicates = df.where(df.duplicated(features,keep=False)==True).dropna(how='all')
        df_incomplete = df.where(df.duplicated(features,keep=False)==False).dropna(how='all')
        df_mean = pd.DataFrame(columns= df.columns)
        for index,values in df_duplicates.drop_duplicates(features).iterrows():
            if len(df_duplicates.loc[index:index+2]) == len(volume_list):
                mean_error_abs =df_duplicates.loc[index:index+2,'%error'].abs().mean()
                mean_error = df_duplicates.loc[index:index+2,'%error'].mean()
                df_duplicates.loc[index,'%error'] = -mean_error_abs
                df_duplicates.loc[index, 'volume'] ='mean_neg'+str(volume_list)
                df_duplicates.loc[index, nan_columns]= 'NaN'
                df_duplicates.loc[index+1,'%error'] = mean_error
                df_duplicates.loc[index+1, 'volume'] ='mean'+str(volume_list)
                df_duplicates.loc[index+1, nan_columns]= 'NaN'
                df_mean = pd.concat([df_mean,df.loc[index:index+2],df_duplicates.loc[[index,index+1]]])
                
            else:
                df_incomplete = pd.concat([df_incomplete,df_duplicates.loc[index:index+2]]).drop_duplicates()
        df_mean = pd.concat([df_mean,df_incomplete])
        df_mean = df_mean.reset_index(drop=True)    
    return df_mean


def compare_optimization(dfs, df_names, volume_list = [1000,500,300], colours = ['#FF5733','#6495ED','#C9292D']):
    """
    This function crates a figure that compares different optimization techniques for a specific pipette. The figure contains
    two plots with the mean percentage error vs iteration and the time to aspirate a 1000uL vs iteration. 
    Args:
        dfs (list):  List of pandas.DataFrame containing data from optimization that will be compared
        df_names (list): List of strings that will be used to distinguish between different techniques.
        volume_list (list): List of volumes as int that were used during the optimization 
        colours (list): List of colour names as str that will be used to compare between different DataFrames
    """

    #Process dfs in dicitonary
    dfs_dict = {}
    
    for i, dfi in enumerate(dfs):
        dfs_dict[df_names[i]]={'full':calculate_means(dfi)}
    
    #Select different volumes
    for key in dfs_dict.keys():
        dfi = dfs_dict[key]['full']
        dfs_dict[key][str(volume_list[0])]= dfi.where(dfi.volume.astype('str')==str(volume_list[0])).dropna(how='all')
        dfs_dict[key][str(volume_list[1])]= dfi.where(dfi.volume.astype('str')==str(volume_list[1])).dropna(how='all')
        dfs_dict[key][str(volume_list[2])]= dfi.where(dfi.volume.astype('str')==str(volume_list[2])).dropna(how='all')
        dfs_dict[key]['mean'+str(volume_list)] = dfi.loc[(dfi['volume'] == 'mean'+ str(volume_list))].dropna(how='all')
        dfs_dict[key]['iteration_min_error'] = dfi.loc[(dfi['volume'] =='mean_neg'+str(volume_list))].dropna(how='all').loc[25:].copy()[['iteration','%error']].abs().sort_values('%error').iloc[0,0]
    fig = make_subplots(rows=2, cols=1)

   
    box_size_dict = {'error':{},'time':{},'iteration':{}}

    for i, key in enumerate(dfs_dict.keys()):
        dfi = dfs_dict[key]['full'].copy()
        dfimean = dfs_dict[key]['mean'+str(volume_list)].copy()
        # dfimin = dfi.where(dfi['iteration']==dfs_dict[key]['iteration_min_error']).copy().dropna(how='all').iloc[:3]

        fig.add_trace(
            go.Scatter(x=dfimean['iteration'], y=dfimean['%error'],mode='lines+markers',  line=dict(color= colours[i]),marker=dict(color= colours[i])), 
            row=1, col=1            
        )

        fig.add_trace(
            go.Scatter(x=dfimean['iteration'], y=dfimean['time_asp_1000'],mode='lines+markers',  line=dict(color= colours[i]), marker=dict(color= colours[i])),
            row=2, col=1
        )
        
        # fig.add_trace(
        #     go.Scatter(x=dfimin['iteration'], y=dfimin['%error'],mode='markers', marker=dict(color= ['#e60909','#0ea103','#000000'], symbol='x')),
        #     row=1, col=1
        # )
        
        iteration_min_error_index= dfimean.where(dfimean.iteration==dfs_dict[key]['iteration_min_error']).dropna(how='all').index[0]
        if box_size_dict['error'] == {}:
            box_size_dict['error'] =  dfimean.loc[:,'%error'].abs().max()*0.07
        else:
            if box_size_dict['error'] < dfimean.loc[:,'%error'].abs().max()*0.07:
                box_size_dict['error'] =  dfimean.loc[:,'%error'].abs().max()*0.07

        if box_size_dict['time'] == {}:
            box_size_dict['time'] =  dfimean.loc[:,'time_asp_1000'].max()*0.05
        else:
            if box_size_dict['time'] < dfimean.loc[:,'time_asp_1000'].max()*0.05:
                box_size_dict['time'] =  dfimean.loc[:,'time_asp_1000'].max()*0.05

        if box_size_dict['iteration'] == {}:
            box_size_dict['iteration'] =  dfimean.loc[:,'iteration'].max()*0.02
        else:
            if box_size_dict['iteration'] < dfimean.loc[:,'iteration'].max()*0.02:
                box_size_dict['iteration'] =  dfimean.loc[:,'iteration'].max()*0.02


    
    for i, key in enumerate(dfs_dict.keys()):
        dfi = dfs_dict[key]['full'].copy()
        dfimean = dfs_dict[key]['mean'+str(volume_list)].copy()

        iteration_min_error_index= dfimean.where(dfimean.iteration==dfs_dict[key]['iteration_min_error']).dropna(how='all').index[0]
        
        fig.add_shape(type="rect",
            x0=dfs_dict[key]['iteration_min_error']-box_size_dict['iteration'], y0=dfimean.loc[iteration_min_error_index,'%error']- box_size_dict['error'], x1=dfs_dict[key]['iteration_min_error']+box_size_dict['iteration'], y1=dfimean.loc[iteration_min_error_index,'%error']+ box_size_dict['error'],
            line=dict(color= colours[i],
            width=2),
            opacity=1,
            fillcolor = "rgba(0,0,0,0)"
        )
        
        iteration_min_error_index= dfimean.where(dfimean.iteration==dfs_dict[key]['iteration_min_error']).dropna(how='all').index[0]
        fig.add_shape(type="rect",
            x0=dfs_dict[key]['iteration_min_error']-box_size_dict['iteration'], y0=dfimean.loc[iteration_min_error_index,'time_asp_1000']-box_size_dict['time'], x1=dfs_dict[key]['iteration_min_error']+box_size_dict['iteration'], y1=dfimean.loc[iteration_min_error_index,'time_asp_1000']+box_size_dict['time'],
            line=dict(color= colours[i],
            width=2),
            opacity=1,
            fillcolor = "rgba(0,0,0,0)",
            row=2, col=1
        )

    fig.add_hline(y=0, line_dash="solid",fillcolor="black",opacity=1, line_width=1,row=1, col=1)
    fig.add_hline(y=5, line_dash="dash",fillcolor="black",opacity=1, line_width=1, row=1, col=1)
    fig.add_hline(y=-5, line_dash="dash",fillcolor="black",opacity=1, line_width=1, row=1, col=1)

    min_value = round(min(fig.to_dict()['data'][0]['y'])/10)*10-5
    max_value = round(max(fig.to_dict()['data'][0]['y'])/10)*10
    if max_value <10 :
        max_value=10

    fig.update_layout(width =500 , height = 500, plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor = 'rgba(255,255,255,255)', margin = {'t':80}, showlegend=False, xaxis  = {'dtick': 1})

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black',ticks='outside',showticklabels=False,mirror=True,row=1,col=1)
    fig.update_yaxes(range =  [min_value, max_value],showline=True, linewidth=1, linecolor='black',ticks='outside',mirror=True,title_text= 'Mean percentage error<br>of transfer [%]',row=1,col=1,tickfont= dict(size=16), titlefont=dict(size=20))
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black',ticks='outside',mirror=True,title_text='Iteration' ,row=2,col=1,tickfont= dict(size=16), titlefont=dict(size=20),  dtick =1)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black',ticks='outside',mirror=True,title_text= 'Time to transfer<br>1000 µL [s]',row=2,col=1,tickfont= dict(size=16), titlefont=dict(size=20))
    return fig
    

In [ ]:
#Cell will return figures containing the comparison plots for all viscous liquids for a specific pipette and set of experiments (ML, Human and FA)

pipette = 'Opentrons' # Pipette used 
folder = '' #path where image will be saved
list_names= '_vs_'.join(['ML','H'])#,'FA'])

for i in ['204','505','817','1275']:
    df_H = pd.read_csv(rf'C:\Users\quijanovelascop\OneDrive - A STAR\Documents\GitHub\LiqTransferOptimizer\{pipette}\Human\Optimization\Viscosity_std_{i}_{pipette}_HO.csv')
    dfML= pd.read_csv(rf'C:\Users\quijanovelascop\OneDrive - A STAR\Documents\GitHub\LiqTransferOptimizer\{pipette}\MOBO\Optimization\Viscosity_std_{i}_{pipette}_MOBO.csv')
   
    #only use with Sartorius pipette 
    # dfFA= pd.read_csv(rf'C:\Users\quijanovelascop\OneDrive - A STAR\Documents\GitHub\LiqTransferOptimizer\{pipette}\Fully_autoamted_optimization\Optimization\Viscosity_std_{i}_Sartorious_FA_MOBO.csv')
   
    y = compare_optimization([dfML,df_H],['ML','H'])
    y.write_image(folder+pipette+'_'+list_names+f'_{i}.svg')
    y

